# Capital Gains Calculator

Given trades inputs from IBKR, determine capital gains from lots after a transfer.


## 1. Derivation of the final formula

Assuming absolute values, then the output for the Post-Transfer P&L for a sale is
$$ \begin{aligned}
\text{Post-Transfer P\&L} &= \text{Proceeds} - \text{Commission} - \text{Transfer Cost} \\
\implies \text{Post-Transfer P\&L} &= \text{Realized P\&L} + \text{Basis} - \text{Quantity} \times \text{Transfer Price}
\end{aligned} $$


In [ ]:
# set the file location
trade_filepath = "./data/raw/[INSERT_FILENAME_HERE].csv"
transfer_filepath = "./data/raw/[INSERT_FILENAME_HERE].csv"


## 2. Read the data

In [ ]:
# install prereqs if not installed
# %pip install pandas --quiet

# standard imports
import re
import pandas as pd

# util imports
from util.read_without_statement_rows import read_without_statement_rows
from util.remove_commas import remove_commas
from util.add_parent_row_index import add_parent_row_index


In [ ]:
# generate the raw dfs
raw_trade_df = read_without_statement_rows(trade_filepath)
raw_transfer_df = read_without_statement_rows(transfer_filepath)


## 3. Cleaning the data

### 3.1. Get Lot Transfer Data
`raw_transfer_df` will contain lots that were transferred to the new accounts. Columns I will need *per lot*:
- Transferred Lot Primary Key:
  - Currency
  - Symbol
  - Date/Time
  - Quantity
  - Purchase Price
- Transfer Price

To get transfer price, I will need to extract parent rows for transfers to get $\text{Transfer Price} = \frac{\text{Market Value}}{\text{Quantity}}$. I will then need to left join parent rows and transferred lots based in their row order.

#### 3.1.1. Filter for Parent Rows and Transferred Lots

First, separate `raw_transfer_df` into two: `transferred_lot_df`, and `transfer_order_df`.

In [ ]:
# create two new dfs based on parents and child rows
transferred_lot_df = raw_transfer_df[raw_transfer_df["Symbol"] == "Transferred Lot:"]
transfer_order_df = raw_transfer_df[raw_transfer_df["Asset Category"] == "Stocks"]


#### 3.1.2. Filter for Useful Transfer Columns
Next, keep and rename the useful columns for each dataframe.

Transferred Lot Columns:
- Row Number (this is stored in the index, and not explicitly stated in code)
- Buy Date
- Buy Quantity
- Buy Price (AKA `"Xfer Price"`)

Transfer Order Columns: 
- Row Number (this is stored in the index, and not explicitly stated in code)
- Currency
- Symbol
- Transfer Date
- Transfer Quantity
- Transfer Market Value

In [ ]:
# set the columns to keep for each and their column names
transferred_lot_cols = {
    "Date"          : "lot_date",       # lot pk
    "Qty"           : "lot_quantity",   # lot pk
    "Xfer Price"    : "lot_buy_price"   # lot pk
}

transfer_order_cols = {
    "Currency"      : "lot_currency",   # lot pk
    "Symbol"        : "lot_symbol",     # lot pk
    "Date"          : "transfer_date",
    "Qty"           : "transfer_quantity",
    "Market Value"  : "transfer_market_value"
}


In [ ]:
# apply column filtering and naming as defined above
transferred_lot_df = transferred_lot_df[transferred_lot_cols.keys()]
transferred_lot_df = transferred_lot_df.rename(transferred_lot_cols, axis = 1)
transfer_order_df = transfer_order_df[transfer_order_cols.keys()]
transfer_order_df = transfer_order_df.rename(transfer_order_cols, axis = 1)


#### 3.1.3. Convert to Correct Data Types

Convert numeric columns to numeric data types so calculations don't cause errors down the line.

In [ ]:
# define numeric columns
transfer_numeric_cols = [
    "lot_quantity", 
    "lot_buy_price", 
    "transfer_quantity",
    "transfer_market_value"
]

# define date cols
transfer_date_cols = ["lot_date", "transfer_date"]


In [ ]:
for col in transfer_numeric_cols:
    if col in transferred_lot_df.columns:
        transferred_lot_df[col] = transferred_lot_df[col].apply(remove_commas)
        transferred_lot_df[col] = transferred_lot_df[col].astype("float")
    if col in transfer_order_df.columns:
        transfer_order_df[col] = transfer_order_df[col].apply(remove_commas)
        transfer_order_df[col] = transfer_order_df[col].astype("float")

for col in transfer_date_cols:
    if col in transferred_lot_df.columns:
        transferred_lot_df[col] = pd.to_datetime(transferred_lot_df[col])
    if col in transfer_order_df.columns:
        transfer_order_df[col] = pd.to_datetime(transfer_order_df[col])


In [ ]:
transferred_lot_df.head(5)


In [ ]:
transfer_order_df.head(5)


### 3.2. Get Closed Lot Data

#### 3.2.1. Filter for Parent Rows and Closed Lots
Separate `raw_trade_df` into two: `closed_lot_df`, and `sell_order_df`.

In [ ]:
# create two new dfs based on parents and child rows
closed_lot_df = raw_trade_df[raw_trade_df["DataDiscriminator"] == "ClosedLot"]
sell_order_df = raw_trade_df[raw_trade_df["DataDiscriminator"] == "Trade"]


#### 3.2.2. Filter for Useful Closed Lot Columns
`raw_trades_df` will contain the lots which are closed (i.e. sold). 

Closed Lot Columns:
- Row Number (this is stored in the index, and not explicitly stated in code)
- Symbol
- Buy Date
- Buy Quantity
- Buy Price (AKA `"T. Price"`)
- Realized P&L
- Basis

Sell Order Columns:
- Row Number (this is stored in the index, and not explicitly stated in code)
- Sell Date

In [ ]:
# set the useful columns for closed lots (and their new names)
closed_lot_cols = {
    "Symbol"        : "lot_symbol",          # pk
    "Date/Time"     : "lot_date",            # pk
    "Quantity"      : "lot_quantity",        # pk
    "T. Price"      : "lot_buy_price",       # pk
    "Realized P/L"  : "lot_realized_pnl",    # Realized P&L
    "Basis"         : "lot_basis",           # Basis
    "Currency"      : "lot_currency"         # Currency (just in case)
}

sell_order_cols = {
    "Date/Time"     : "lot_sell_date"        # sell date
}


In [ ]:
# create the closed_lot dataframe with above-defined columns
closed_lot_df = closed_lot_df[closed_lot_cols.keys()]
closed_lot_df = closed_lot_df.rename(closed_lot_cols, axis = 1)
sell_order_df = sell_order_df[sell_order_cols.keys()]
sell_order_df = sell_order_df.rename(sell_order_cols, axis = 1)


#### 3.2.3. Convert to Correct Data Types
Convert numeric columns to numeric data types so calculations don't cause errors down the line.

In [ ]:
# define the numeric columns
closed_lot_numeric_cols = [
    "lot_quantity",
    "lot_buy_price",
    "lot_realized_pnl",
    "lot_basis"
]

# define date cols
closed_lot_date_cols = ["lot_date", "lot_sell_date"]


In [ ]:
for col in closed_lot_numeric_cols:
    if col in closed_lot_df.columns:
        closed_lot_df[col] = closed_lot_df[col].apply(remove_commas)
        closed_lot_df[col] = closed_lot_df[col].astype("float")
    if col in sell_order_df.columns:
        sell_order_df[col] = sell_order_df[col].apply(remove_commas)
        sell_order_df[col] = sell_order_df[col].astype("float")

for col in closed_lot_date_cols:
    if col in closed_lot_df.columns:
        closed_lot_df[col] = pd.to_datetime(closed_lot_df[col])
    if col in sell_order_df.columns:
        sell_order_df[col] = pd.to_datetime(sell_order_df[col])


In [ ]:
closed_lot_df.head(5)


In [ ]:
sell_order_df.head(5)


### 3.3. Save cleaned data

In [ ]:
sell_order_df.to_csv("./data/cleaned/sell_orders.csv")
closed_lot_df.to_csv("./data/cleaned/closed_lots.csv")
transfer_order_df.to_csv("./data/cleaned/transfer_orders.csv")
transferred_lot_df.to_csv("./data/cleaned/transferred_lots.csv")


## 4. Joining the data

### 4.1. Joining transferred lots and transfer orders

#### 4.1.1. Determine which transfer order corresponds to which transferred lot


In [ ]:
# calculate the parent rows
transferred_lot_df = add_parent_row_index(transferred_lot_df, transfer_order_df)


In [ ]:
transferred_lot_df.head(5)


#### 4.1.2. Join transferred lots and transfer orders


In [ ]:
# join the dfs
transfer_df = transfer_order_df.merge(
    transferred_lot_df,
    left_index = True,
    right_on = "parent_index"
)


In [ ]:
transfer_df.head(5)


### 4.2. Joining closed lots and sell dates

#### 4.2.1. Determine which sell date corresponds to which closed lot

In [ ]:
# calculate the parent rows
closed_lot_df = add_parent_row_index(closed_lot_df, sell_order_df)


In [ ]:
closed_lot_df.head(5)


#### 4.2.2. Join closed lots and sell dates

In [ ]:
# join the dfs
sell_df = closed_lot_df.merge(
    sell_order_df,
    left_on = "parent_index",
    right_index = True
)


In [ ]:
sell_df.head(5)


### 4.3. Joining transfers and closed lots

In [ ]:
# define the index for lots
merge_cols = [
    "lot_symbol",
    "lot_date",
    "lot_buy_price",
    "lot_currency"
]


In [ ]:
# TODO: Delete this once a better solution presents itself
# Round `lot_buy_price` 
# to prevent different s.f. levels between trades and transfers to affect join
lot_buy_price_decimal_places = 6
transfer_df["lot_buy_price"] = transfer_df["lot_buy_price"].apply(
    lambda n: round(n, lot_buy_price_decimal_places)
)
sell_df["lot_buy_price"] = sell_df["lot_buy_price"].apply(
    lambda n: round(n, lot_buy_price_decimal_places)
)


In [ ]:
output_df = transfer_df.merge(
    sell_df,
    how = "left",
    on = merge_cols,
    suffixes = ["_t", "_cl"],

)


In [ ]:
output_df.head(5)


### 4.4. Save the joined data

In [ ]:
transfer_df.to_csv("./data/cleaned/joined/transfers.csv")
sell_df.to_csv("./data/cleaned/joined/sells.csv")


## 5. Calculate Output

Assuming absolute values, then the output for the Post-Transfer P&L for a sale is
$$ \begin{aligned}
\text{Post-Transfer P\&L} &= \text{Proceeds} - \text{Commission} - \text{Transfer Cost} \\
\implies \text{Post-Transfer P\&L} &= \text{Realized P\&L} + \text{Basis} - \text{Quantity Sold} \times \text{Transfer Price}
\end{aligned} $$

### 5.1. Calculate the transfer price

- Transfer Price (where $\text{Transfer Price} = \frac{\text{Transfer Market Value}}{\text{Transfer Quantity}}$)

In [ ]:
output_df.loc[:, "transfer_price"] = output_df["transfer_market_value"] \
    / output_df["transfer_quantity"]


In [ ]:
output_df.head(5)


### 5.2. Pre-transfer Profit/Loss

#### 5.2.1. Calculate pre-transfer profit/loss

Assuming absolute values, then the output for the Pre-Transfer P&L for a sale is
$$ \begin{aligned}
\text{Pre-Transfer P\&L} &= \text{Transfer Market Value} - \text{Commission} - \text{Purchase Cost} \\
\implies \text{Pre-Transfer P\&L} &= \text{Quantity Sold} \times \text{Transfer Price} - \text{Basis}
\end{aligned} $$

In [ ]:
output_df.loc[:, "pre_transfer_pnl"] = (
    (
        output_df["lot_quantity_cl"] *
        output_df["transfer_price"]
    ) - output_df["lot_basis"]
)

# round
output_df["pre_transfer_pnl"] = output_df["pre_transfer_pnl"].apply(
    lambda n: round(n, 5)
)


#### 5.2.2. Determine whether pre-transfer was long-term or short-term

Consider it short term if $\text{Transfer Date} - \text{Purchase Date} < 12 \text{ Months}$

In [ ]:
# calculate the time elapsed between sale and transfer
output_df.loc[:, "time_between_buy_and_transfer"] = (
    output_df["transfer_date"] - output_df["lot_date"]
)
output_df.loc[:, "time_since_transfer"] = pd.to_timedelta(
    output_df["time_between_buy_and_transfer"])


In [ ]:
# create a lt/st dictionary to map true/false values to their time codes.
lt_st_dict = {True: "LT", False: "ST"}


In [ ]:
output_df.loc[:, "pre_transfer_is_long_term"] = \
    (output_df["time_between_buy_and_transfer"].dt.total_seconds() > (365.25 * 24 * 60 * 60)).map(lt_st_dict)


### 5.3. Post-transfer Profit/Loss

#### 5.3.1. Calculate the post-transfer profit/loss

Assuming absolute values, then the output for the Post-Transfer P&L for a sale is
$$ \begin{aligned}
\text{Post-Transfer P\&L} &= \text{Proceeds} - \text{Commission} - \text{Transfer Cost} \\
\implies \text{Post-Transfer P\&L} &= \text{Realized P\&L} + \text{Basis} - \text{Quantity Sold} \times \text{Transfer Price}
\end{aligned} $$

In [ ]:
output_df.loc[:, "post_transfer_pnl"] = (
    output_df["lot_realized_pnl"] + 
    output_df["lot_basis"] - (
        output_df["lot_quantity_cl"] * 
        output_df["transfer_price"]
    )
)

# round
output_df["post_transfer_pnl"] = output_df["post_transfer_pnl"].apply(
    lambda n: round(n, 5)
)


#### 5.3.2. Determine whether post-transfer is a long-term or short-term sale

Consider it short term if $\text{Sell Date} - \text{Transfer Date} < 12 \text{ Months}$

In [ ]:
# calculate the time elapsed between sale and transfer
output_df.loc[:, "time_since_transfer"] = (
    output_df["lot_sell_date"] - output_df["transfer_date"]
)
output_df.loc[:, "time_since_transfer"] = pd.to_timedelta(output_df["time_since_transfer"])


In [ ]:
# create a lt/st dictionary to map true/false values to their time codes.
lt_st_dict = {
    True    : "LT",
    False   : "ST"
}


In [ ]:
output_df.loc[:, "post_transfer_is_long_term"] = \
    (output_df["time_since_transfer"].dt.total_seconds() > (365.25 * 24 * 60 * 60)).map(lt_st_dict)


In [ ]:
output_df[output_df["post_transfer_pnl"].notna()]


In [ ]:
output_df.columns


## 6. Save the output

### 6.1. Rename the columns for output

In [ ]:
# columns to save to output
output_cols = {
    "lot_currency"          : "lot_currency",
    "lot_symbol"            : "lot_symbol",
    "lot_date"              : "lot_date",
    "lot_buy_price"         : "lot_buy_price",
    "lot_basis"             : "lot_basis",
    "transfer_date"         : "transfer_date",
    "transfer_price"        : "transfer_price",
    "lot_quantity_cl"       : "sell_quantity",
    "lot_realized_pnl"      : "sell_realised_pnl",
    "lot_sell_date"         : "sell_date",
    "post_transfer_pnl"     : "post_transfer_realised_pnl",
    "time_since_transfer"   : "post_transfer_time_elapsed",
    "is_long_term"          : "post_transfer_is_long_term"
}


In [ ]:
output_df = output_df[output_cols.keys()]
output_df = output_df.rename(output_cols, axis = 1)


### 6.2. Save to CSV

In [ ]:
output_df.to_csv("./data/out/output.csv")
